In [1]:
import tushare as ts
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import merge
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import time
import datetime

# 1、注册接口

In [2]:
#ts.set_token('4a7caeea1e5e4718dac4386b6e4f7bfefdafd2f9634f9e71e6ffaaef')
#ts.set_token('d6fe22750410e124c057a41d1c8031d684b8e0d2f1b96b694db0ea2a') #15112994875
#ts.set_token('ca4d2743af67b24bcbc3b14f09fa349b9877bc893b28706041600a9e') #492207670@qq.com
#ts.set_token('86f7cdc6aad080403432e395cee208218866d4f091063e69d8c8d755') #714553504@qq.com
#ts.set_token('0242bb7d53bef05e3ce8e426c7b4ac86edf30ca779d0b0e48d4a7882') #1755347637@qq.com
#ts.set_token('8f68a2ccc10e31a9d66ae27e6930bdb8cb99a6eb75b8d5e8da0a495b')  #26098495@qq.com
ts.set_token('9f5585ac3e8a5fd2e4be500723102060c609881d22e2b9f9436d3409')  #2364480207@qq.com
#ts.set_token('b1b07de72441487cc9bdced5c94fab3765605559a50106e3e621a95c')  #763690782@qq.com
#ts.set_token('01e7c939be853b6e8cc8b2cfb19facd42733e68062485b5d8c8a3a38')  #3509036093@qq.com
#ts.set_token('52341d49b9b7616a7f98976bcb41d29d21e723a68015afd6c97f781b')  #13433245820
#ts.set_token('a06020cba02aa5cf32bc461b4e529af5852de1e7c19e5a22a9f7c118')  #roelwf134@163.com

#ts.set_token('029060c40e4a6b9b2ffc3e55a18f6690b9e3ba9c77ab8a1ece015bda')  #2978373708@qq.com
#ts.set_token('e6215c86b05f42be90a84255b65924c7e3e64cd8b189f09de97a9a90')  #huelw124@sina.com
#ts.set_token('89f9642565ae1134ca836fe089382335b46b880a626c2ec42ade6d87')  #1516836216@qq.com
#ts.set_token('2b0108ead201cfff4f868f17f5f4269ac9617d740e24aea8c7ae5983') #holdetf@sina.com

In [3]:
pro = ts.pro_api()

In [4]:
start = time.time()

In [5]:
# 插入py文件，调用里面的函数
from Stock_Interface import build_finance_year
from Stock_Interface.build_finance_year import to_stock_list, to_cal, to_stock_company, read_year, build_daily, read_cal_t, build_stock_newyear
from Stock_Interface.build_finance_year import to_adjfactor, read_adjfactor, build_adjfactor, to_newyear

# 2、更新基础信息

In [6]:
to_stock_list(pro)
to_cal(pro)
to_stock_company(pro)

# 3、更新每日价格信息

In [8]:
# 刷新最细stock数据
years = ['2022']

for b_year in years:
    build_daily(pro, b_year)

ok!


In [9]:
print('finish:%.4f'%(time.time() - start))

finish:135.3733


In [10]:
df_cal_t = read_cal_t('20220101', '20221231')
count = int( df_cal_t.count() )
print(count)

242


# 4、重构ALL日数据，按时间段

In [11]:
%%time
df_stock_daily = build_stock_newyear()
df_stock_daily.info

Wall time: 7min 8s


<bound method DataFrame.info of             ts_code  trade_date   open   high    low  close  pre_close  \
0         600807.SH    20000102  22.10  22.70  21.85  21.90      21.99   
1         600807.SH    20010102  22.10  22.70  21.85  21.90      21.99   
2         600871.SH    20010102   6.60   6.74   6.60   6.70       6.51   
3         000655.SZ    20010102  18.00  18.17  17.93  18.16      17.89   
4         600778.SH    20010102  15.03  16.38  15.03  15.94      14.92   
...             ...         ...    ...    ...    ...    ...        ...   
11199940  871553.BJ    20220128   5.37   5.54   5.37   5.53       5.50   
11199941  871642.BJ    20220128  15.93  16.50  15.60  15.63      15.96   
11199942  871981.BJ    20220128  41.01  42.69  40.60  41.77      41.20   
11199943  872925.BJ    20220128  22.23  22.23  21.31  21.78      21.30   
11199944  689009.SH    20220128  60.65  61.06  59.60  60.81      60.60   

          change  pct_chg        vol       amount  
0          -0.09  -0.4100  

In [12]:
df_stock_daily.tail(6)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
11199939,871396.BJ,20220128,12.19,12.32,11.70,12.03,12.15,-0.12,-0.9877,561.51,674.673
11199940,871553.BJ,20220128,5.37,5.54,5.37,5.53,5.50,0.03,0.5455,1377.70,758.779
11199941,871642.BJ,20220128,15.93,16.50,15.60,15.63,15.96,-0.33,-2.0677,2480.00,3931.593
11199942,871981.BJ,20220128,41.01,42.69,40.60,41.77,41.20,0.57,1.3835,3033.26,12670.273
11199943,872925.BJ,20220128,22.23,22.23,21.31,21.78,21.30,0.48,2.2535,323.36,699.169
11199944,689009.SH,20220128,60.65,61.06,59.60,60.81,60.60,0.21,0.3465,10422.22,63002.207


In [13]:
df_stock_daily[(df_stock_daily.ts_code == '601398.SH')].tail(20)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
11109724,601398.SH,20211231,4.63,4.65,4.63,4.63,4.64,-0.01,-0.2155,959022.15,444544.835
11114395,601398.SH,20220104,4.64,4.67,4.63,4.67,4.63,0.04,0.8639,1688756.35,785143.329
11119065,601398.SH,20220105,4.66,4.72,4.65,4.71,4.67,0.04,0.8565,2839546.21,1332859.587
11123737,601398.SH,20220106,4.70,4.73,4.68,4.70,4.71,-0.01,-0.2123,1903143.59,894943.768
11128413,601398.SH,20220107,4.69,4.75,4.68,4.74,4.70,0.04,0.8511,1939649.55,916673.916
11133089,601398.SH,20220110,4.74,4.76,4.72,4.75,4.74,0.01,0.2110,1583293.03,750737.778
11137769,601398.SH,20220111,4.73,4.77,4.73,4.75,4.75,0.00,0.0000,1670262.97,794398.037
11142449,601398.SH,20220112,4.75,4.75,4.70,4.72,4.75,-0.03,-0.6316,1857315.19,876687.477
11147133,601398.SH,20220113,4.71,4.78,4.71,4.73,4.72,0.01,0.2119,1793135.00,851827.613
11151815,601398.SH,20220114,4.74,4.74,4.67,4.68,4.73,-0.05,-1.0571,2251970.35,1057017.144


# 5、生成权重因子

https://www.pythonheidong.com/blog/article/235000/eb07100a752acc29bece/

In [31]:
#df_adj = pro.adj_factor(ts_code='', trade_date='20210114')

In [14]:
# 刷新最细stock数据
years = [#'2001','2002','2003','2004','2005','2006','2007','2008','2009','2010',
         #'2011','2012','2013','2014','2015','2016','2017','2018','2019','2020',
        '2022'
        ]

for b_year in years:
    build_adjfactor(pro, b_year)

ok!


In [15]:
print('finish:%.4f'%(time.time() - start))

finish:782.9947


In [16]:
# 合成数据 adjfactor
build_finance_year.build_stock_newyear_adjfactor()

Empty DataFrame
Columns: []
Index: [(000001.SZ, 20220124, 111.921), (000001.SZ, 20220125, 111.921), (000001.SZ, 20220126, 111.921), (000001.SZ, 20220127, 111.921), (000001.SZ, 20220128, 111.921)]


           ts_code  trade_date  open  high   low  close  pre_close  change  \
6541698  601939.SH    20211231  5.86  5.87  5.84   5.86       5.87   -0.01   
6541699  601939.SH    20220104  5.85  5.92  5.85   5.91       5.86    0.05   
6541700  601939.SH    20220105  5.91  6.04  5.90   6.03       5.91    0.12   
6541701  601939.SH    20220106  6.00  6.09  6.00   6.05       6.03    0.02   
6541702  601939.SH    20220107  6.06  6.15  6.01   6.14       6.05    0.09   
6541703  601939.SH    20220110  6.13  6.18  6.08   6.12       6.14   -0.02   
6541704  601939.SH    20220111  6.11  6.17  6.09   6.14       6.12    0.02   
6541705  601939.SH    20220112  6.14  6.14  6.03   6.08       6.14   -0.06   
6541706  601939.SH    20220113  6.08  6.16  6.06   6.11       6.08    0.03   
6541707  601939.SH    

In [17]:
from Stock_Interface.read_finance_year import read_stock, read_newyear, read_stockmore
data = read_newyear()
data[(data.ts_code == '601939.SH') & (data.trade_date > 20190615)].head(20)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,adj_factor_x,adj_factor_y,close1
6541077,601939.SH,20190617,7.23,7.28,7.21,7.26,7.22,0.04,0.5540,441973.80,320474.137,1.693,1.952,6.297
6541078,601939.SH,20190618,7.25,7.37,7.23,7.35,7.26,0.09,1.2397,905587.48,662772.825,1.693,1.952,6.375
6541079,601939.SH,20190619,7.45,7.49,7.40,7.46,7.35,0.11,1.4966,1053841.96,784250.021,1.693,1.952,6.470
6541080,601939.SH,20190620,7.45,7.62,7.42,7.54,7.46,0.08,1.0724,1316689.80,993897.229,1.693,1.952,6.540
6541081,601939.SH,20190621,7.50,7.56,7.38,7.41,7.54,-0.13,-1.7241,1631764.98,1214047.171,1.693,1.952,6.427
6541082,601939.SH,20190624,7.41,7.45,7.38,7.44,7.41,0.03,0.4049,753380.03,558724.228,1.693,1.952,6.453
6541083,601939.SH,20190625,7.44,7.44,7.25,7.34,7.44,-0.10,-1.3441,1142915.06,838613.811,1.693,1.952,6.366
6541084,601939.SH,20190626,7.31,7.44,7.30,7.38,7.34,0.04,0.5450,729670.64,538394.956,1.693,1.952,6.401
6541085,601939.SH,20190627,7.39,7.46,7.38,7.41,7.38,0.03,0.4065,788634.73,584583.026,1.693,1.952,6.427
6541086,601939.SH,20190628,7.41,7.44,7.35,7.44,7.41,0.03,0.4049,654995.25,484191.673,1.693,1.952,6.453


In [18]:
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,adj_factor_x,adj_factor_y,close1
0,600807.SH,20010102,22.10,22.70,21.85,21.90,21.99,-0.09,-0.4100,3578.10,7928.222,2.387,6.536,7.998
1,600807.SH,20010103,21.91,21.91,21.21,21.75,21.90,-0.15,-0.6800,2761.36,5917.409,2.387,6.536,7.943
2,600807.SH,20010104,21.90,22.00,21.50,21.54,21.75,-0.21,-0.9700,1569.55,3393.122,2.387,6.536,7.867
3,600807.SH,20010105,21.30,21.79,21.23,21.32,21.54,-0.22,-1.0200,1860.33,3970.499,2.387,6.536,7.786
4,600807.SH,20010108,21.40,21.40,20.62,20.78,21.32,-0.54,-2.5300,2424.00,5120.334,2.387,6.536,7.589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10912878,301217.SZ,20220128,18.99,20.30,18.18,18.30,21.70,-3.40,-15.6682,681867.46,1297597.033,1.000,1.000,18.300
10912879,688171.SH,20220127,34.00,40.05,34.00,35.44,28.68,6.76,23.5704,134102.36,488225.946,1.000,1.000,35.440
10912880,688171.SH,20220128,31.10,31.20,28.11,28.72,35.44,-6.72,-18.9616,80654.63,239363.024,1.000,1.000,28.720
10912881,688270.SH,20220127,69.00,69.00,53.85,56.20,61.88,-5.68,-9.1791,122591.72,719259.346,1.000,1.000,56.200
